In [49]:
import pandas as pd
import os
from pybaseball import statcast, pitching_stats, batting_stats, playerid_lookup, statcast_batter, statcast_pitcher

from pull_data import *
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from stat_miner import *

In [51]:
from rotowire import get_game_lineups

In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


url = "https://www.rotowire.com/baseball/daily-lineups.php"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

data_pitching = []
data_batter = []
team_type = ''

for e in soup.select('.lineup__box ul li'):
    if team_type != e.parent.get('class')[-1]:
        order_count = 1
        team_type = e.parent.get('class')[-1]

    if e.get('class') and 'lineup__player-highlight' in e.get('class'):
        data_pitching.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'lineup_throws':e.span.get_text(strip=True)
        })
    elif e.get('class') and 'lineup__player' in e.get('class'):
        data_batter.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'pos': e.div.get_text(strip=True),
            'batting_order':order_count,
            'lineup_bats':e.span.get_text(strip=True)
        })
        order_count+=1

df_pitching = pd.DataFrame(data_pitching)
df_batter = pd.DataFrame(data_batter)

In [53]:
df_batter

,date,game_time,pitcher_name,team,pos,batting_order,lineup_bats
0,2025-06-01,1:05 PM ET,L. Nootbaar,Cardinals,LF,1,L
1,2025-06-01,1:05 PM ET,Masyn Winn,Cardinals,SS,2,R
2,2025-06-01,1:05 PM ET,B. Donovan,Cardinals,2B,3,L
3,2025-06-01,1:05 PM ET,W. Contreras,Cardinals,1B,4,R
4,2025-06-01,1:05 PM ET,Ivan Herrera,Cardinals,DH,5,R
...,...,...,...,...,...,...,...
265,2025-06-01,7:10 PM ET,Andy Pages,Dodgers,LF,5,R
266,2025-06-01,7:10 PM ET,Tommy Edman,Dodgers,CF,6,S
267,2025-06-01,7:10 PM ET,Max Muncy,Dodgers,3B,7,L
268,2025-06-01,7:10 PM ET,E. Hernandez,Dodgers,2B,8,R


In [57]:
df_batter['team_id'] = df_batter['team'].map(get_team_mapping())

In [71]:
team_ids = df_batter['team_id'].unique()
team_ids = [int(team_id) for team_id in team_ids]
print(team_ids)

[138, 140, 145, 110, 111, 144, 158, 143, 133, 141, 108, 114, 115, 121, 137, 146, 116, 118, 139, 117, 113, 112, 142, 136, 120, 109, 134, 135, 147, 119]


In [66]:
[player.split(' ')[-2:] for player in statsapi.roster(143).split('\n')]

[['Alec', 'Bohm'],
 ['Brandon', 'Marsh'],
 ['Bryce', 'Harper'],
 ['Bryson', 'Stott'],
 ['Carlos', 'Hernández'],
 ['Cristopher', 'Sánchez'],
 ['Edmundo', 'Sosa'],
 ['J.T.', 'Realmuto'],
 ['Jesús', 'Luzardo'],
 ['Joe', 'Ross'],
 ['Johan', 'Rojas'],
 ['Jordan', 'Romano'],
 ['Kyle', 'Schwarber'],
 ['Matt', 'Strahm'],
 ['Max', 'Kepler'],
 ['Max', 'Lazar'],
 ['Nick', 'Castellanos'],
 ['Orion', 'Kerkering'],
 ['Rafael', 'Marchán'],
 ['Ranger', 'Suárez'],
 ['Seth', 'Johnson'],
 ['Taijuan', 'Walker'],
 ['Tanner', 'Banks'],
 ['Trea', 'Turner'],
 ['Weston', 'Wilson'],
 ['Zack', 'Wheeler'],
 ['']]

In [73]:
def map_player_ids(team_ids):
    player_names = []
    player_ids = []
    for team_id in team_ids:
        roster = statsapi.roster(team_id)
        for player in roster:
            name = player.split(' ')[-2:]
            if len(name) == 1:
                continue
            player_id = playerid_lookup(name[0], name[1], fuzzy=True)
            player_names.append(player_id)
            player_ids.append(player_id)

    player_id_mapping = pd.DataFrame({'name': player_names, 'player_id': player_ids})
    player_id_mapping.to_csv('player_id_mapping.csv', index=False)

In [74]:
map_player_ids(team_ids)

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

ConnectionError: HTTPSConnectionPool(host='statsapi.mlb.com', port=443): Max retries exceeded with url: /api/v1/teams/133/roster?rosterType=active&season=2025 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x129f638c0>: Failed to establish a new connection: [Errno 51] Network is unreachable'))